In [1]:
import random
import torch
import numpy as np
from torch import nn,optim
from model import RN
from functions import from_batch
from torch.autograd import Variable

In [2]:
# define hyperparameters
batch_size = 32
embed_size = 64
en_hidden_size = 32
mlp_hidden_size = 256
epochs = 30

# optional: if you're starting off from a previous model
startoff=10

In [4]:
# load dataset
# with open('datasets/train_qa1.txt') as f:
with open('/home/irteam/users/data/bAbI/bAbI/datasets/train_10k.txt') as f:
    lines=f.readlines()
if len(lines)%batch_size==0:
    num_batches = int(len(lines)/batch_size)
else:
    num_batches = int(len(lines)/batch_size)

# load vocabulary
word2idx = np.load('word2idx.npy').item()
idx2word = np.load('idx2word.npy').item()

In [5]:
vocab_size = len(word2idx)

In [6]:
# rn = torch.load('saved/rn_%d.pth' %startoff)
rn = RN(vocab_size, embed_size, en_hidden_size, mlp_hidden_size)
if torch.cuda.is_available():
    rn = rn.cuda()
opt = optim.Adam(rn.parameters(),lr=2e-4)
criterion = nn.CrossEntropyLoss()

In [8]:
# for validation
def validate(rn, val_set):
    S, Q, A = from_batch(val_set)
    out = rn(S, Q)
    O = torch.max(out,1)[1].cpu().data.numpy().squeeze()
    score = np.array(O==A,int)
    total = len(score)
    correct = sum(score)
    return total,correct

with open('/home/irteam/users/data/bAbI/bAbI/datasets/test_qa1.txt') as f:
# with open('test_1,2,3.txt') as f:
    val_set=f.readlines()[:batch_size]

In [17]:
batch[0].strip().split('::')[0].split('\t')

['127 28 107 155 97',
 '95 65 155 26 115',
 '127 28 107 155 90',
 '95 103 43 155 26',
 '166 121 155 26',
 '166 132 155 26',
 '102 99 107 155 62',
 '95 72 20 107 155 161',
 '95 99 107 155 14',
 '166 98 21 155 26',
 '95 28 107 155 161',
 '127 28 107 155 62',
 '127 103 43 155 144',
 '166 28 107 155 161',
 '95 72 20 107 155 97',
 '166 132 155 26',
 '102 121 155 144 115',
 '127 7 107 155 97',
 '102 28 107 155 14',
 '102 72 20 107 155 90']

In [9]:
# training
# for epoch in range(epochs):
for epoch in range(10):
    random.shuffle(lines) # shuffle lines
    for i in range(num_batches):
#     for i in range(5):
        opt.zero_grad()
#         print(i)
        batch = lines[i*batch_size:(i+1)*batch_size]
        S,Q,A = from_batch(batch)
        out = rn(S,Q)
        A = Variable(torch.LongTensor(A))
        if torch.cuda.is_available():
            A = A.cuda()
        loss = criterion(out,A)
        loss.backward()
        opt.step()
        if i % 20==0:
            print("loss for %d/%d: %1.3f" % (i,num_batches,loss.data[0]))
    print("loss for epoch %d: %1.3f" % (epoch,loss.data[0]))
    model_name = 'rn_%d.pth' % (epoch+1+startoff)
#     if i % 5==0:
#         torch.save(obj=rn,f='saved/'+model_name)
    total, correct = validate(rn, val_set)
    print("Validation score: ",correct*1.0/total)

RuntimeError: size mismatch at /home1/irteam/users/mjchoi/Downloads/pytorch/torch/lib/THC/generic/THCTensorMathBlas.cu:243

In [ ]:
model_name = 'rn_%d.pth' % (epoch+1)
torch.save(obj=rn,f='saved/'+model_name)

In [ ]:
# validation
# load dataset
with open('test.txt') as f:
    lines=f.readlines()
num_batches = int(len(lines)/batch_size)

# load vocabulary
word2idx = np.load('word2idx.npy').item()
idx2word = np.load('idx2word.npy').item()

rn = torch.load('saved/rn_5.pth')
if torch.cuda.is_available():
    rn = rn.cuda()

# for epoch in range(epochs):
total = 0
correct = 0
for epoch in range(1):
    random.shuffle(lines) # shuffle lines
    for i in range(num_batches):
#     for i in range(5):
        opt.zero_grad()
#         print(i)
        batch = lines[i*batch_size:(i+1)*batch_size]
        S,Q,A = from_batch(batch)
        out = rn(S,Q)
        # out : [batch x vocab_size]
        O = torch.max(out,1)[1].cpu().data.numpy()
        score = np.array(O==A, int)
        total+=len(score)
        correct+=sum(score)

In [ ]:
np.array(O.squeeze() == A,dtype=int)

In [ ]:
min(S[1])

In [ ]:
sum(S[2])

In [ ]:
q

In [ ]:
a